# TetCTF 2020 - (Crypto) yaecc

Challenge's code: [yaecc.py](./yaecc.py)

We are given an ECDSA service using NIST-p256 curve with Dual_EC_Drbg. Dual_EC_Drbg indeed has a backdoor, we tried to looking for it but ... :< Anyway, now we recall the equation

$$H(m_i) = s_ik_i - xr_i \; (mod \; q) \; ; \forall i$$

where $q$ is curve order and $x$ is private key. We can get rid of $x$ by using the first equation ($i = 0$)

$$H(m_0) = s_0k_0 - xr_0 \; (mod \; q)$$

$$x = [s_ik_i - H(m_i)]r_i^{-1} = [s_0k_0 - H(m_0)]r_0^{-1} \; (mod \; q) \; ; \forall i \neq 0$$

$$k_i - (s_0s_i^{-1}r_ir_0^{-1})k_0 - [H(m_i) - H(m_0)r_ir_0^{-1}]s_i^{-1} = 0  \; (mod \; q) \; ; \forall i \neq 0$$

We now have equation with only 2 unknowns

$$k_i + A_ik_0 + B_i = 0  \; (mod \; q)$$

and also a system with $k_i < q$

$$\left\{\begin{array}{l}
k_0\\ 
k_1 = -A_1k_0 + z_1q - B_1\\ 
\cdots\\ 
k_{n-1} = -A_{n-1}k_0 + z_{n-1}q - B_{n-1}\\
\end{array}\right.$$

Note that $\forall i,$ $k_i$ is "small" (240 bits compare to 256 bits of $q$). LLL will be useful here.

Consider the following lattice (spanned by rows):

$$\Lambda =
\begin{array}{c c}
    \begin{array}{c}
      -k_0 \\ z_1 \\ \vdots \\ z_{n-1} \\ -1
    \end{array}\hspace{-1em}
    &
    \left(
      \begin{array}{@{} c c c c c @{}}
        -1 & A_1 & \cdots & A_{n-1} & 0 \\
        0 & q & \cdots & 0 & 0   \\
        \vdots & \vdots & \ddots  & \vdots & \vdots \\
        0 & 0 & \cdots & q & 0 \\
        0 & B_1 & \cdots & B_{n-1} & T \\
      \end{array}
    \right) \\
     & \begin{array} {@{} c c c c c @{}}
        \hspace{0.2cm}k_0 & k_1 & \cdots & k_{n-1} & T  
      \end{array} \\
\end{array}.$$

Apply LLL to the above matrix, we will obtain vector

$$(k_0, k_1, \cdots, k_{n-1}, T)$$

as a short vector in the lattice since all $k_i$ must be small. Here we call $T$ a "scale factor" and set $T = q \; / \; 2^{16}$ for better result.

In [1]:
from sage.all import *
from sock import Sock
from hashlib import sha256
import os
from Crypto.Util.number import bytes_to_long

In [2]:
EC = EllipticCurve(
    GF(0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff),
    [-3, 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b]
)
n = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551
Zn = Zmod(n)
G = EC((0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296,
        0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5))
P = G
Q = EC((0xc97445f45cdef9f0d3e05e1e585fc297235b82b5be8ff3efca67c59852018192,
        0xb28ef557ba31dfcbdd21ac46e2a91e3c304f44cb87058ada2cb815151e610046))
pubkey = EC((50590195252452518804028762265927043036734617153869060607666882619539230027822,
        36611353725619757431858072740028832533174535444901899686884685372270344860185))

class DualEcDrbg(object):
    """
    Dual Elliptic Curve Deterministic Random Bit Generator
    """

    def __init__(self, seed):
        self.s = ZZ(bytes_to_long(seed))

    def next_bits(self):
        """
        Transit to the next state and output 240 bits as specified in the
        document above.
        """
        self.s = ZZ((self.s * P)[0])
        return ZZ((self.s * Q)[0]) & (2 ** 240 - 1)
    
def sign(private_key, message, rand):
    """
    Implementation of ECDSA signature generation algorithm.

    Arguments:
        private_key: an element of Zn
        message: a byte array/string
        rand: should be an instance of DualEcDrbg

    Output:
        a pair of elements of Zn as signature
    """
    z = Zn(ZZ(sha256(message).hexdigest(), 16))
    k = Zn(rand.next_bits())
    assert k != 0
    K = ZZ(k) * G
    r = Zn(K[0])
    assert r != 0
    s = (z + r * private_key) / k
    assert s != 0
    return r, s

In [3]:
# prepare data
data = []
for _ in range(50):
    s = Sock("207.148.119.58 7777")
    m = eval(s.read_line())
    sig = eval(s.read_line())
    data.append((m, sig))
    s.close()
print len(data)
print data[0]
with open("data.txt", "w") as f:
    f.write(str(data))

50
(692494932444911938281358884854815034654651722854898313688926780816615518L, (104657486054259951439910822571627111745999399075656361550091463134738835849548L, 63541373551002245103978740867757192637590832602921383256029372141567837298207L))


In [4]:
size = 20
m = []
Ai = [-1]
Bi = [0]
r0, s0 = map(Zn, data[0][1])
z0 = Zn(ZZ(sha256(str(data[0][0])).hexdigest(), 16))
for i in range(size):
    message, sig = data[i+1]
    ri, si = map(Zn, sig)
    zi = z = Zn(ZZ(sha256(str(message)).hexdigest(), 16))
    A = - (s0 * ri) / (r0 * si)
    B = (z0 * ri) / (si * r0) - zi / si
    Ai.append(A)
    Bi.append(B)
Ai.append(0)
Bi.append(n//2^16)
m.append(Ai)
for i in range(size):
    m.append([0]*(i+1) + [n] + [0]*(size-i))
m.append(Bi)
m = Matrix(ZZ, m)

mLLL = m.LLL()

for irow, row in enumerate(mLLL):
    k0 = Zn(row[0])
    d = (s0*k0-z0)/r0
    if pubkey == ZZ(d)*G:
        print "Found priv key:", d
        break
    k0 = Zn(-row[0])
    d = (s0*k0-z0)/r0
    if pubkey == ZZ(d)*G:
        print "Found priv key:", d
        break
msg2 = b"I am admin"
rand = DualEcDrbg(os.urandom(16))
sig = sign(ZZ(d), msg2, rand)
s = Sock("207.148.119.58 7777")
s.send_line(str(sig))
ret = s.read_all()
print ret

Found priv key: 18945449323768766527988564133889338261474122476740549153948717492043782641361
1376570555983669930554257310292832223285285800384039643976688889272512497
(13042820918839700822473378869719375490659982400051070318856576257855626371887, 58391984902778699759177516921815648158618818661500250702192593529072534281221)
TetCTF{_0oops____Sm4ll_k_ru1n3d_th3_p4rty_}

